## Basis Swap Par Spreads

The goal of the project is to compute the par spreads of a pool of basis swaps, i.e. those contracts in which two floating libor rates, with different tenors, are exchanged. The par spread is that margin which has to be paid/received on top of the floating rate with the shorter tenor.

The input parameters are:

* The par rate of a set of Overnight Index Swaps (i.e. their market quotation)
* 3 sets of forward libor rates and their fixing dates, one for the Libor 1M, one for the Libor 3M and one for the Libor 6M.
* The static data of the pool of swaps (nominal, maturity, tenor of the first leg, tenor of the second leg)

Numerical results must be presented to the examining committee during a presentation in which the candidates will also explain the theoretical framework for multi-curve evaluation.

In [1]:
from datetime import date
today = date(2019, 10, 31)

In [2]:
ois_quotes = [
    {'maturity': 1, 'rate': 0.00106},
    {'maturity': 2, 'rate': 0.00114},
    {'maturity': 3, 'rate': 0.00115},
    {'maturity': 4, 'rate': 0.00117},
    {'maturity': 5, 'rate': 0.00119},
    {'maturity': 6, 'rate': 0.00121},
    {'maturity': 7, 'rate': 0.00122},
    {'maturity': 8, 'rate': 0.00124},
    {'maturity': 9, 'rate': 0.00128},
    {'maturity': 10, 'rate': 0.00131},
    {'maturity': 11, 'rate': 0.00135},
    {'maturity': 12, 'rate': 0.00138},
    {'maturity': 15, 'rate': 0.00152},
    {'maturity': 18, 'rate': 0.00166},
    {'maturity': 21, 'rate': 0.00184},
    {'maturity': 24, 'rate': 0.00206},
    {'maturity': 36, 'rate': 0.00344},
    {'maturity': 48, 'rate': 0.00543},
    {'maturity': 60, 'rate': 0.00756},
    {'maturity': 72, 'rate': 0.00967},
    {'maturity': 84, 'rate': 0.01162},
    {'maturity': 96, 'rate': 0.0134},
    {'maturity': 108, 'rate': 0.01502},
    {'maturity': 120, 'rate': 0.01649},
    {'maturity': 132, 'rate': 0.01776},
    {'maturity': 144, 'rate': 0.01888},
    {'maturity': 180, 'rate': 0.02137},
    {'maturity': 240, 'rate': 0.02322},
    {'maturity': 300, 'rate': 0.02389},
    {'maturity': 360, 'rate': 0.02416},
]

In [3]:
euribor_1m = [
    {'date': date(2019, 10, 31), 'rate': 0.002},
    {'date': date(2020, 4, 30), 'rate': 0.00431709159584514},
    {'date': date(2020, 10, 30), 'rate': 0.00513437204910293},
    {'date': date(2021, 4, 30), 'rate': 0.00595155807365439},
    {'date': date(2021, 10, 30), 'rate': 0.00676883852691218},
    {'date': date(2022, 4, 30), 'rate': 0.00758611898016997},
    {'date': date(2022, 10, 30), 'rate': 0.00840339943342776},
    {'date': date(2023, 4, 30), 'rate': 0.00922058545797923},
    {'date': date(2023, 10, 30), 'rate': 0.010037865911237},
    {'date': date(2024, 4, 30), 'rate': 0.0108550519357885},
    {'date': date(2024, 10, 30), 'rate': 0.0116723323890463},
    {'date': date(2025, 4, 30), 'rate': 0.0124895184135977},
    {'date': date(2025, 10, 30), 'rate': 0.0133067988668555},
    {'date': date(2026, 4, 30), 'rate': 0.0141240793201133},
    {'date': date(2026, 10, 30), 'rate': 0.0149413597733711},
    {'date': date(2027, 4, 30), 'rate': 0.0157585457979226},
    {'date': date(2027, 10, 30), 'rate': 0.0165758262511804},
    {'date': date(2028, 4, 30), 'rate': 0.0173930122757318},
    {'date': date(2028, 10, 30), 'rate': 0.0182102927289896},
    {'date': date(2029, 4, 30), 'rate': 0.0190274787535411},
    {'date': date(2029, 10, 30), 'rate': 0.0198447592067989},
    {'date': date(2030, 4, 30), 'rate': 0.0206620396600567},
    {'date': date(2030, 10, 30), 'rate': 0.0214793201133144},
    {'date': date(2031, 4, 30), 'rate': 0.0222965061378659},
    {'date': date(2031, 10, 30), 'rate': 0.0231137865911237},
    {'date': date(2032, 4, 30), 'rate': 0.0239309726156752},
    {'date': date(2032, 10, 30), 'rate': 0.024748253068933},
    {'date': date(2033, 4, 30), 'rate': 0.0255654390934844},
    {'date': date(2033, 10, 30), 'rate': 0.0263827195467422},
    {'date': date(2034, 4, 30), 'rate': 0.0267},
]

euribor_3m = [
    {'date': date(2019, 10, 31), 'rate': 0.0035},
    {'date': date(2020, 4, 30), 'rate': 0.00431709159584514},
    {'date': date(2020, 10, 30), 'rate': 0.00513437204910293},
    {'date': date(2021, 4, 30), 'rate': 0.00595155807365439},
    {'date': date(2021, 10, 30), 'rate': 0.00676883852691218},
    {'date': date(2022, 4, 30), 'rate': 0.00758611898016997},
    {'date': date(2022, 10, 30), 'rate': 0.00840339943342776},
    {'date': date(2023, 4, 30), 'rate': 0.00922058545797923},
    {'date': date(2023, 10, 30), 'rate': 0.010037865911237},
    {'date': date(2024, 4, 30), 'rate': 0.0108550519357885},
    {'date': date(2024, 10, 30), 'rate': 0.0116723323890463},
    {'date': date(2025, 4, 30), 'rate': 0.0124895184135977},
    {'date': date(2025, 10, 30), 'rate': 0.0133067988668555},
    {'date': date(2026, 4, 30), 'rate': 0.0141240793201133},
    {'date': date(2026, 10, 30), 'rate': 0.0149413597733711},
    {'date': date(2027, 4, 30), 'rate': 0.0157585457979226},
    {'date': date(2027, 10, 30), 'rate': 0.0165758262511804},
    {'date': date(2028, 4, 30), 'rate': 0.0173930122757318},
    {'date': date(2028, 10, 30), 'rate': 0.0182102927289896},
    {'date': date(2029, 4, 30), 'rate': 0.0190274787535411},
    {'date': date(2029, 10, 30), 'rate': 0.0198447592067989},
    {'date': date(2030, 4, 30), 'rate': 0.0206620396600567},
    {'date': date(2030, 10, 30), 'rate': 0.0214793201133144},
    {'date': date(2031, 4, 30), 'rate': 0.0222965061378659},
    {'date': date(2031, 10, 30), 'rate': 0.0231137865911237},
    {'date': date(2032, 4, 30), 'rate': 0.0239309726156752},
    {'date': date(2032, 10, 30), 'rate': 0.024748253068933},
    {'date': date(2033, 4, 30), 'rate': 0.0255654390934844},
    {'date': date(2033, 10, 30), 'rate': 0.0263827195467422},
    {'date': date(2034, 4, 30), 'rate': 0.0272},
]

euribor_6m = [
    {'date': date(2019, 10, 31), 'rate': 0.005},
    {'date': date(2020, 4, 30), 'rate': 0.0058},
    {'date': date(2020, 10, 30), 'rate': 0.0066},
    {'date': date(2021, 4, 30), 'rate': 0.0074},
    {'date': date(2021, 10, 30), 'rate': 0.0082},
    {'date': date(2022, 4, 30), 'rate': 0.009},
    {'date': date(2022, 10, 30), 'rate': 0.0098},
    {'date': date(2023, 4, 30), 'rate': 0.0106},
    {'date': date(2023, 10, 30), 'rate': 0.0114},
    {'date': date(2024, 4, 30), 'rate': 0.0122},
    {'date': date(2024, 10, 30), 'rate': 0.013},
    {'date': date(2025, 4, 30), 'rate': 0.0138},
    {'date': date(2025, 10, 30), 'rate': 0.0146},
    {'date': date(2026, 4, 30), 'rate': 0.0154},
    {'date': date(2026, 10, 30), 'rate': 0.0162},
    {'date': date(2027, 4, 30), 'rate': 0.017},
    {'date': date(2027, 10, 30), 'rate': 0.0178},
    {'date': date(2028, 4, 30), 'rate': 0.0186},
    {'date': date(2028, 10, 30), 'rate': 0.0194},
    {'date': date(2029, 4, 30), 'rate': 0.0202},
    {'date': date(2029, 10, 30), 'rate': 0.021},
    {'date': date(2030, 4, 30), 'rate': 0.0218},
    {'date': date(2030, 10, 30), 'rate': 0.0226},
    {'date': date(2031, 4, 30), 'rate': 0.0234},
    {'date': date(2031, 10, 30), 'rate': 0.0242},
    {'date': date(2032, 4, 30), 'rate': 0.025},
    {'date': date(2032, 10, 30), 'rate': 0.0258},
    {'date': date(2033, 4, 30), 'rate': 0.0266},
    {'date': date(2033, 10, 30), 'rate': 0.0274},
    {'date': date(2034, 4, 30), 'rate': 0.0282},
]

In [4]:
basis_swaps = [
    {'nominal': 1000000, 'maturity': 12, 'first_tenor': 1, 'second_tenor': 3},
    {'nominal': 1000000, 'maturity': 12, 'first_tenor': 3, 'second_tenor': 6},
    {'nominal': 1000000, 'maturity': 12, 'first_tenor': 1, 'second_tenor': 6},
    {'nominal': 1000000, 'maturity': 24, 'first_tenor': 1, 'second_tenor': 3},
    {'nominal': 1000000, 'maturity': 24, 'first_tenor': 3, 'second_tenor': 6},
    {'nominal': 1000000, 'maturity': 24, 'first_tenor': 1, 'second_tenor': 6},
    {'nominal': 1000000, 'maturity': 60, 'first_tenor': 1, 'second_tenor': 3},
    {'nominal': 1000000, 'maturity': 60, 'first_tenor': 3, 'second_tenor': 6},
    {'nominal': 1000000, 'maturity': 60, 'first_tenor': 1, 'second_tenor': 6},
    {'nominal': 1000000, 'maturity': 90, 'first_tenor': 1, 'second_tenor': 3},
    {'nominal': 1000000, 'maturity': 90, 'first_tenor': 3, 'second_tenor': 6},
    {'nominal': 1000000, 'maturity': 90, 'first_tenor': 1, 'second_tenor': 6},
    {'nominal': 1000000, 'maturity': 120, 'first_tenor': 1, 'second_tenor': 3},
    {'nominal': 1000000, 'maturity': 120, 'first_tenor': 3, 'second_tenor': 6},
    {'nominal': 1000000, 'maturity': 120, 'first_tenor': 1, 'second_tenor': 6},
]